In [2]:
import matplotlib.pyplot as plt
from typing import List
import seaborn as sns
import os
import nnts
import nnts.data
import nnts.experiments
import nnts.models
import nnts.torch.preprocessing as preprocessing
import nnts.torch.models
import nnts.torch.trainers as trainers
import nnts.metrics
import nnts.torch.data
import nnts.torch.datasets
import nnts.loggers
import nnts.pandas
from nnts import utils
import covs 

sns.set()

/Users/garethdavies/Development/workspaces/nnts/venv/lib/python3.11/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


ModuleNotFoundError: No module named 'utils'

In [ ]:
df, *_ = nnts.pandas.read_tsf(
    "traffic_weekly_dataset.tsf",
    "https://zenodo.org/records/4656135/files/traffic_weekly_dataset.zip",
)

In [ ]:
ARTICLE_PATH = "nb-results/figures"
results_path = "nb-results"
data_path = "data"
dataset = "tourism"
datafile_path = os.path.join(data_path, covs.file_map[dataset])
model_name = "unrolled-lstm"
PATH = os.path.join(results_path, model_name, dataset)
df_orig, *_ = nnts.pandas.read_tsf(datafile_path)
nnts.loggers.makedirs_if_not_exists(ARTICLE_PATH)
nnts.loggers.makedirs_if_not_exists(PATH)

In [ ]:
df_orig, metadata = nnts.pandas.load(
    dataset, data_path, metadata_filename="seg-lstm-monash.json"
)

params = utils.Hyperparams()
#params.training_method = utils.Hyperparams.TrainingMethod.FREE_RUNNING
splitter = nnts.pandas.LastHorizonSplitter()

In [ ]:
scenario_list: List[nnts.experiments.CovariateScenario] = []

# Add the baseline scenarios
for seed in [42, 43, 44, 45, 46]:
    scenario_list.append(
        nnts.experiments.CovariateScenario(metadata.prediction_length, error=0.0, covariates=0, seed=seed)
    )

## Models for full forecast horizon with covariates
for covariates in [1, 2, 3]:
    for error in covs.errors[metadata.dataset]:
        scenario_list.append( 
            nnts.experiments.CovariateScenario(
                metadata.prediction_length, error, covariates=covariates
            )
        )

scenario_list.append( 
    nnts.experiments.CovariateScenario(
        metadata.prediction_length, 0, covariates=3, skip=1
    )
)

In [ ]:
for scenario in scenario_list[:1]:
    nnts.torch.datasets.seed_everything(scenario.seed)
    df, scenario = covs.prepare(df_orig.copy(), scenario)
    split_data = splitter(df, metadata.context_length, metadata.prediction_length)
    trn_dl, val_dl, test_dl = nnts.data.create_trn_val_test_dataloaders(
        split_data,
        metadata,
        scenario,
        params,
        nnts.torch.data.TorchTimeseriesDataLoaderFactory(),
    )
    logger = nnts.loggers.WandbRun(
        project=f"{model_name}-{metadata.dataset}",
        name=scenario.name,
        config={
            **params.__dict__,
            **metadata.__dict__,
            **scenario.__dict__,
        },
        path=PATH
    )

    net = nnts.torch.models.UnrolledLSTM(
        nnts.torch.models.LinearModel,
        params,
        preprocessing.masked_mean_abs_scaling,
        scenario.covariates + 1,
    )
    trner = trainers.TorchEpochTrainer(
        nnts.trainers.TrainerState(), 
        net, 
        params, 
        metadata, 
        os.path.join(PATH, f"{scenario.name}.pt"),
    )
    logger.configure(trner.events)
    evaluator = trner.train(trn_dl, val_dl)
    handle = net.decoder.register_forward_hook(logger.log_activations)
    y_hat, y = evaluator.evaluate(
        test_dl, scenario.prediction_length, metadata.context_length, hooks=handle
    )
    handle.remove()
    test_metrics = nnts.metrics.calc_metrics(
        y, y_hat, metadata.freq, metadata.seasonality
    )
    logger.log(test_metrics)
    logger.finish()

In [ ]:
test_metrics

In [ ]:
csv_aggregator = nnts.pandas.CSVFileAggregator(PATH, "results")
results = csv_aggregator()

In [ ]:
df_list = covs.add_y_hat(df, y_hat, scenario.prediction_length)
sample_preds = covs.plot(df_list, scenario.prediction_length)

In [ ]:
univariate_results = results.loc[
    (results["covariates"] == 0)
    & (results["prediction_length"] == metadata.prediction_length),
    ["smape", "mape", "rmse", "mae"],
]

univariate_results.mean(), univariate_results.std(), univariate_results.count()

In [ ]:
cols = ["dataset", "error", "pearson", "covariates", "prediction_length", "smape"]

In [ ]:
results.loc[
    (results["covariates"] > 0)
    & (results["error"].isin([0.000000, 0.4714285714285714, 1.65])),
    cols,
].sort_values(by=["covariates", "error"])

In [ ]:
sorted(results["error"].unique().tolist())